In [2]:
from zipline.data import bundles 
import pandas as pd
from zipline.data.data_portal import DataPortal

In [3]:
import os

from zipline.utils.run_algo import load_extensions
from zipline.utils.calendar_utils import register_calendar, get_calendar

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

In [4]:
from zipline.pipeline.loaders import USEquityPricingLoader

In [5]:
data = bundles.load('adjusted-dse-csvdir')
trading_calendar = get_calendar('XDSE')

In [6]:
data_portal = DataPortal(
    data.asset_finder,
    trading_calendar= trading_calendar,
    first_trading_day=data.equity_daily_bar_reader.first_trading_day,
    equity_daily_reader=data.equity_daily_bar_reader,
    adjustment_reader=data.adjustment_reader
)

In [7]:
def get_pricing(assets, start_date, end_date, fields='close', trading_calendar=None):
    """
    Approximate Quantopian function `get_pricing` available
    in online environment at quantopian.com
    """
    if trading_calendar is None:
        trading_calendar = get_calendar("NYSE")
    
    # Set the given start and end dates to Timestamps.
    end_date = pd.Timestamp(end_date, tz='utc')
    start_date = pd.Timestamp(start_date, tz='utc')

    # Get the locations of the start and end dates
    sessions = trading_calendar.sessions_in_range(start_date, end_date)
    bar_count = len(sessions)
    
    # Get identifiers for asset symbols
    equities = data.asset_finder.lookup_symbols(assets, start_date)

    # return the historical data for the given window
    return data_portal.get_history_window(
                            assets=equities,
                            end_dt=end_date,
                            bar_count=bar_count,
                            frequency='1d',
                            field=fields,
                            data_frequency='daily')

In [8]:
df = get_pricing(["ACFL"], fields='open')

TypeError: get_pricing() missing 2 required positional arguments: 'start_date' and 'end_date'

In [15]:
df

,Equity(1 [ACFL])
2018-09-10 00:00:00+00:00,44.427
2018-09-11 00:00:00+00:00,45.289
2018-09-12 00:00:00+00:00,45.289
2018-09-13 00:00:00+00:00,48.222
2018-09-16 00:00:00+00:00,49.603
...,...
2021-12-26 00:00:00+00:00,30.000
2021-12-27 00:00:00+00:00,29.400
2021-12-28 00:00:00+00:00,28.600
2021-12-29 00:00:00+00:00,31.000
